### Parámetros de entorno y rutas

In [0]:
# [01][1] Parámetros de entorno y rutas 
# secentraliza la configuración

dbutils.widgets.text("CATALOGO",               "workspace")
dbutils.widgets.text("ESQUEMA_BRONCE",         "bronze_mb")
dbutils.widgets.text("PREFIJO_TABLA",          "mb_")

dbutils.widgets.text("VOLUMEN_CATALOGO",       "workspace")
dbutils.widgets.text("VOLUMEN_ESQUEMA",        "default")
dbutils.widgets.text("VOLUMEN_LANDING",        "landing")

# Opciones de lectura CSV (parametrizables para flexibilizar pruebas)
dbutils.widgets.dropdown("CSV_HEADER",         "true", ["true","false"])
dbutils.widgets.text("CSV_DELIMITADOR",        ",")
dbutils.widgets.text("CSV_COMILLA",            '"')
dbutils.widgets.text("CSV_ESCAPE",             "\\")
dbutils.widgets.text("CSV_ENCODING",           "UTF-8")

# Selección de archivos
dbutils.widgets.text("ARCHIVO_GLOB",           "*.csv")
dbutils.widgets.text("EXCLUIR_REGEX",          r".*/validation/.*")

# Control del micro-batch en modo: poller
dbutils.widgets.text("TAMANO_MICROBATCH",      "1")
dbutils.widgets.text("SEGUNDOS_ESPERA",        "0")

# ---- Lectura parámetros
CATALOGO         = dbutils.widgets.get("CATALOGO").strip()
ESQUEMA_BRONCE   = dbutils.widgets.get("ESQUEMA_BRONCE").strip()
PREFIJO_TABLA    = dbutils.widgets.get("PREFIJO_TABLA").strip()

V_CAT            = dbutils.widgets.get("VOLUMEN_CATALOGO").strip()
V_ESQ            = dbutils.widgets.get("VOLUMEN_ESQUEMA").strip()
V_LANDING        = dbutils.widgets.get("VOLUMEN_LANDING").strip()

CSV_HEADER       = dbutils.widgets.get("CSV_HEADER").strip()
CSV_DELIM        = dbutils.widgets.get("CSV_DELIMITADOR")
CSV_QUOTE        = dbutils.widgets.get("CSV_COMILLA")
CSV_ESCAPE       = dbutils.widgets.get("CSV_ESCAPE")
CSV_ENCODING     = dbutils.widgets.get("CSV_ENCODING")

ARCHIVO_GLOB     = dbutils.widgets.get("ARCHIVO_GLOB").strip()
EXCLUIR_REGEX    = dbutils.widgets.get("EXCLUIR_REGEX").strip()

TAMANO_MICROBATCH= int(dbutils.widgets.get("TAMANO_MICROBATCH"))
SEGUNDOS_ESPERA  = int(dbutils.widgets.get("SEGUNDOS_ESPERA"))

# ---- Rutas y tablas
spark.sql(f"USE CATALOG {CATALOGO}")

RUTA_LANDING = f"/Volumes/{V_CAT}/{V_ESQ}/{V_LANDING}"
T_BRONCE     = f"{CATALOGO}.{ESQUEMA_BRONCE}.{PREFIJO_TABLA}events_bronce"
T_ESTADO     = f"{CATALOGO}.{ESQUEMA_BRONCE}.{PREFIJO_TABLA}archivos_ingeridos"

display({
    "RUTA_LANDING": RUTA_LANDING,
    "TABLA_BRONCE": T_BRONCE,
    "TABLA_ESTADO": T_ESTADO,
    "DELIMITADOR": CSV_DELIM,
    "GLOB": ARCHIVO_GLOB,
    "EXCLUIR_REGEX": EXCLUIR_REGEX,
    "TAMANO_MICROBATCH": TAMANO_MICROBATCH,
    "SEGUNDOS_ESPERA": SEGUNDOS_ESPERA
})


### DDL de tablas

In [0]:
# [01][2] Creo la existencia de las tablas físicas en Delta
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOGO}.{ESQUEMA_BRONCE}")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {T_BRONCE} (
  timestamp     STRING,
  price         STRING,
  user_id       STRING,
  _ingest_ts    TIMESTAMP,
  _source_file  STRING,
  _batch_id     STRING
) USING DELTA
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {T_ESTADO} (
  file_path     STRING,
  processed_at  TIMESTAMP
) USING DELTA
""")

print("[INFO] Tablas listas: Bronce y Estado de archivos.")

- ### Utilidades: listar candidatos y filtrar pendientes

In [0]:
# [01][3] Utilidades de descubrimiento:
import re, fnmatch

def list_archivos_candidatos(dir_base: str, glob_pat: str, excluir_regex: str):
    entradas = dbutils.fs.ls(dir_base)
    archivos = [e.path for e in entradas if e.path.lower().endswith(".csv")]
    candidatos = [p for p in archivos if fnmatch.fnmatch(p.split("/")[-1], glob_pat)]
    if excluir_regex:
        rx = re.compile(excluir_regex)
        candidatos = [p for p in candidatos if not rx.match(p)]
    return sorted(candidatos)

def list_archivos_pendientes(candidatos: list, tabla_estado: str):
    if not candidatos:
        return []
    df_cand = spark.createDataFrame([(p,) for p in candidatos], "file_path STRING")
    ya = spark.table(tabla_estado).select("file_path")
    pend = df_cand.join(ya, "file_path", "left_anti")
    return [r["file_path"] for r in pend.collect()]


###Ingesta de un micro-batch (procesa N archivos por iteración)

In [0]:
# [01][4] Función de ingesta de micro-batch:

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import regexp_extract, current_timestamp, lit

# Esquema raw en Bronce:
SCHEMA_RAW = StructType([
    StructField("timestamp", StringType(), True),
    StructField("price",     StringType(), True),
    StructField("user_id",   StringType(), True),
])

def ingerir_microbatch(file_paths: list, batch_size: int) -> int:
    objetivos = file_paths[:batch_size]
    if not objetivos:
        return 0

    for fp in objetivos:
        df = (spark.read
              .format("csv")
              .option("header", CSV_HEADER)
              .option("delimiter", CSV_DELIM)
              .option("quote", CSV_QUOTE)
              .option("escape", CSV_ESCAPE)
              .option("encoding", CSV_ENCODING)
              .schema(SCHEMA_RAW)
              .load(fp)
              .withColumn("_source_file", lit(fp))
              .withColumn("_batch_id", regexp_extract(lit(fp), r'.*/([^/]+)\.csv$', 1))
              .withColumn("_ingest_ts", current_timestamp())
             )

        # Insercion filas en Bronce (append-only, raw)
        (df.select("timestamp","price","user_id","_ingest_ts","_source_file","_batch_id")
           .write.format("delta").mode("append").saveAsTable(T_BRONCE))

        (spark.createDataFrame([(fp,)], "file_path STRING")
              .withColumn("processed_at", current_timestamp())
              .write.format("delta").mode("append").saveAsTable(T_ESTADO))

        print(f"[OK] Ingerido → {fp}")

    return len(objetivos)

### Ejecución: una pasada

In [0]:
# [01][5] Ejecución controlada:
# - SEGUNDOS_ESPERA = 0  => recorre pendientes una vez y termina (estilo available-now)
# - SEGUNDOS_ESPERA > 0  => hace polling cada X segundos 

import time

while True:
    candidatos = list_archivos_candidatos(RUTA_LANDING, ARCHIVO_GLOB, EXCLUIR_REGEX)
    pendientes = list_archivos_pendientes(candidatos, T_ESTADO)

    if not pendientes:
        print("[INFO] No hay archivos pendientes en landing.")
    else:
        n = ingerir_microbatch(pendientes, TAMANO_MICROBATCH)
        print(f"[INFO] Procesados en este micro-batch: {n} | Pendientes remanentes: {max(0, len(pendientes)-n)}")

    if SEGUNDOS_ESPERA <= 0:
        break

    time.sleep(SEGUNDOS_ESPERA)
